In [2]:
from google.colab import files
from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np

from datetime import datetime
drive.mount('/content/drive') #/MyDrive/Colab Notebooks/데이터/기상데이터

Mounted at /content/drive


In [19]:
cloud = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/cloud_median.csv', index_col = 0)
wind = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/wind_median.csv', index_col = 0)
ta = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/ta_median.csv', index_col = 0)
rn = pd.read_csv('drive/MyDrive/Colab Notebooks/데이터/기상데이터/rn_median.csv', index_col = 0)

In [38]:
cloud['date'] = pd.to_datetime(cloud['date'])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')

In [39]:
cloud_Gangwon = cloud[cloud['add']=='강원']
cloud_Gyeonggi = cloud[cloud['add']=='경기']
cloud_Gyeongnam = cloud[cloud['add']=='경남']
cloud_Gyeongbuk = cloud[cloud['add']=='경북']
cloud_Gwangju = cloud[cloud['add']=='광주']
cloud_Daegu = cloud[cloud['add']=='대구']
cloud_Daejeon = cloud[cloud['add']=='대전']
cloud_Busan = cloud[cloud['add']=='부산']
cloud_Seoul = cloud[cloud['add']=='서울']
cloud_Sejong = cloud[cloud['add']=='세종']
cloud_Ulsan = cloud[cloud['add']=='울산']
cloud_Incheon = cloud[cloud['add']=='인천']
cloud_Jeonnam = cloud[cloud['add']=='전남']
cloud_Jeonbuk = cloud[cloud['add']=='전북']
cloud_Jeju = cloud[cloud['add']=='제주']
cloud_Chungnam = cloud[cloud['add']=='충남']
cloud_Chungbuk = cloud[cloud['add']=='충북']

In [40]:
days = pd.DataFrame({'date' : pd.date_range(start="2012-02-01",end="2022-10-31")})

In [22]:
cloud_Gangwon

,date,add,stn_id,cloud.ca,cloud.ch,lat,long
0,2012-02-01,강원,114.0,7.0,10.0,37.337495,128.457415
26,2012-02-03,강원,100.0,4.0,30.0,37.902610,127.735700
40,2012-02-04,강원,101.0,7.0,9.0,37.902610,127.946595
56,2012-02-05,강원,100.0,7.0,60.0,37.902610,128.564715
72,2012-02-06,강원,101.0,6.0,60.0,37.902610,127.946595
...,...,...,...,...,...,...,...
50930,2022-10-26,강원,93.0,5.5,60.0,37.947415,127.754490
50946,2022-10-27,강원,104.0,3.0,57.5,37.804580,128.855375
50962,2022-10-28,강원,104.0,7.0,10.0,37.804580,128.855375
50978,2022-10-29,강원,104.0,6.0,10.0,37.804580,128.855375


In [41]:
cloud_Gangwon_outer = pd.merge(cloud_Gangwon, days, on = ['date'], how = 'outer').sort_values(by = 'date')
cloud_Gangwon_outer

,date,add,stn_id,cloud.ca,cloud.ch,lat,long
0,2012-02-01,강원,114.0,7.0,10.0,37.337495,128.457415
3253,2012-02-02,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-02-03,강원,100.0,4.0,30.0,37.902610,127.735700
2,2012-02-04,강원,101.0,7.0,9.0,37.902610,127.946595
3,2012-02-05,강원,100.0,7.0,60.0,37.902610,128.564715
...,...,...,...,...,...,...,...
3249,2022-10-27,강원,104.0,3.0,57.5,37.804580,128.855375
3250,2022-10-28,강원,104.0,7.0,10.0,37.804580,128.855375
3251,2022-10-29,강원,104.0,6.0,10.0,37.804580,128.855375
3252,2022-10-30,강원,104.0,6.0,9.0,37.804580,128.855375


In [47]:
cloud_Gangwon_outer['cloud.ca'] = cloud_Gangwon_outer['cloud.ca'].interpolate(method = 'values')
cloud_Gangwon_outer['cloud.ch'] = cloud_Gangwon_outer['cloud.ch'].interpolate(method = 'values')

In [55]:
cloud_Gangwon_outer['add'] = '강원'
del cloud_Gangwon_outer['stn_id']
del cloud_Gangwon_outer['lat']
del cloud_Gangwon_outer['long']

In [56]:
cloud_Gangwon_outer

,date,add,cloud.ca,cloud.ch
0,2012-02-01,강원,7.0,10.0
3253,2012-02-02,강원,6.0,9.0
1,2012-02-03,강원,4.0,30.0
2,2012-02-04,강원,7.0,9.0
3,2012-02-05,강원,7.0,60.0
...,...,...,...,...
3249,2022-10-27,강원,3.0,57.5
3250,2022-10-28,강원,7.0,10.0
3251,2022-10-29,강원,6.0,10.0
3252,2022-10-30,강원,6.0,9.0


아.. 다 합쳐서 해야 겠네.  
아닌데..? 어차피 나중에 날짜랑 시도로 이어붙이고 정렬만 해주면 되니까, stn_id, lat, long은 이제 필요없잖아.  
ㅇㅇ이게 맞는거 같아